In [1]:
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [2]:
import tensorflow as tf
from skimage.io import imread
from PIL import Image
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices(device_type='GPU')


sample_image = imread(f'Simplified_Data/0/0_40.2288329678609,-124.22826817046267.jpg')
print(sample_image.shape)
batch_size = 20
img_height = 200
img_width = 200

Num GPUs Available:  1
(640, 640, 3)


In [3]:
usaPolyGrid = pkl.load(open("util/usaPolyGrid.pkl",'rb'))
base_path = 'Data/'
grids = np.array(list(usaPolyGrid.keys()),np.int32)

In [ ]:
# class My_Custom_Generator(keras.utils.Sequence):
#     def __init__(self, image_filenames, labels, batch_size) :
#         self.image_filenames = image_filenames
#         self.labels = labels
#         self.batch_size = batch_size
#
#
#     def __len__(self) :
#         return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int32)
#
#
#     def __getitem__(self, idx) :
#         batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
#         batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
#
#         return preprocess_input(np.array([
#             imread(BASE_DIR + '/training_data/' + str(file_name))
#             for file_name in batch_x])), np.array(batch_y)

## Format Data

In [4]:

# ds_train = tf.keras.preprocessing.image_dataset_from_directory(
#     'Data/',
#     labels='inferred',
#     label_mode = 'categorical',
#     color_mode='grayscale',
#     batch_size=64,
#     image_size=(1000, 1000),
#     shuffle=True,
#     seed=123,
#     validation_split=.15,
#     subset="training"
# )
#
# ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
#     'Data/',
#     labels='inferred',
#     label_mode = 'categorical',
#     color_mode='grayscale',
#     batch_size=64,
#     image_size=(1000, 1000),
#     shuffle=True,
#     seed=123,
#     validation_split=.15,
#     subset="validation"
# )

train_ds = tf.keras.utils.image_dataset_from_directory(
    'Simplified_Data',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
    'Simplified_Data',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)



Found 83708 files belonging to 243 classes.
Using 66967 files for training.
Found 83708 files belonging to 243 classes.
Using 16741 files for validation.


In [ ]:
def augment(x,y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return  image,y
train_ds = train_ds.map(augment)

labels = []
for i in range(243):
    labels.append(i)

class_names = val_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")




In [11]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

NameError: name 'train_ds' is not defined

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)


NameError: name 'train_ds' is not defined

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))



In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(10, (3, 3), activation='relu', input_shape=(640, 640, 1)))
### Apply Max pooling, poolsize = 2x2, stride = 2x2 as it will default pool_sizer is strides are automated
#### https://keras.io/api/layers/pooling_layers/max_pooling2d/
# ↓
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Conv2D(10, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Conv2D(10, (3, 3), activation='relu'))
# Unwrap rows into a single row
model.add(tf.keras.layers.Flatten())
model.summary()
# Add Fully connected layer
model.add(tf.keras.layers.Dense(64, activation='relu'))

model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model = Sequential([
#     layers.Conv2D(10,(5,5), activation='relu', input_shape=(img_height, img_width, 3)),
#     layers.MaxPooling2D((5,5)),
#     layers.Conv2D(5, (5, 5), padding='same', activation='relu'),
#     layers.MaxPooling2D((5,5)),
#     layers.Conv2D(10, (5, 5), padding='same', activation='relu'),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(243)
# ])
# layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
# layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
# layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
# layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
# layers.Flatten(),
# layers.Dense(1024),
# layers.Dense(num_classes)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 638, 638, 10)      100       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 319, 319, 10)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 317, 317, 10)      910       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 158, 158, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 156, 156, 10)      910       
_________________________________________________________________
flatten (Flatten)            (None, 243360)            0         
Total params: 1,920
Trainable params: 1,920
Non-trainable params: 0
______________________________________________________

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 638, 638, 10)      100       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 319, 319, 10)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 317, 317, 10)      910       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 158, 158, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 156, 156, 10)      910       
_________________________________________________________________
flatten (Flatten)            (None, 243360)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [6]:
data = None

In [7]:
import os

for x in os.listdir('Train_data_npy/'):
    data = np.load(f'Train_data_npy/{x}', allow_pickle=True)
    all_imgs = []
    all_labels = []
    for img , lable in data:
        all_imgs.append(img)
        all_labels.append( lable)
    data = None
    all_imgs = np.array(all_imgs)
    all_labels = np.array(all_labels)
    plt.imshow(all_imgs[0])
    all_imgs = all_imgs.astype('float32')/255.0
    epochs=10

    history = model.fit(
        all_imgs,
        all_labels,
        validation_data=val_ds,
        epochs=epochs
    )


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-6d8fd586ff3c>", line 5, in <module>
    all_imgs = []
  File "<ipython-input-7-6d8fd586ff3c>", line 5, in <module>
    all_imgs = []
  File "/home/shane/.local/share/JetBrains/Toolbox/apps/DataSpell/ch-0/222.4345.24/plugins/python-ce/helpers/pydev/_pydevd_bundle/pydevd_frame.py", line 880, in trace_dispatch
    stopped_on_plugin = plugin_manager.stop(main_debugger, frame, event, self._args, stop_info, arg, step_cmd)
  File "/home/shane/.local/share/JetBrains/Toolbox/apps/DataSpell/ch-0/222.4345.24/plugins/python-ce/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 169, in stop
    main_debugger.do_wait_suspend(thread, frame, event, arg)
  File "/home/shane/.local/share/JetBrains/Toolbox/apps/DataSpell/ch-0/222.4345.24/plugins/

TypeError: object of type 'NoneType' has no len()

In [4]:
loaded_model = tf.keras.models.load_model('shane_model')
loaded_model.predict(val_ds)

TypeError: __init__() got an unexpected keyword argument 'ignore_class'

In [ ]:
# import matplotlib.pyplot as plt
#
# plt.figure(figsize=(10, 10))
# for images, labels in ds_train.take(1):
#     for i in range(243):
#         ax = plt.subplot(3,81, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")
#

In [ ]:
# from tensorflow.keras import layers
# from tensorflow import keras
#
# model = keras.Sequential([
#     layers.Input((1000, 1000, 1)),
#     layers.Conv2D(16,3,padding='same'),
#     layers.Conv2D(32,3,padding='same'),
#     layers.MaxPool2D(),
#     layers.Flatten(),
#     layers.Dense(10),
# ])

In [ ]:
# model.compile(optimizer=keras.optimizers.Adam(),
#               loss=[
#                   keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               ],
#               metrics=['accuracy'],
#             )
# model.fit(ds_train, epochs=10, verbose=2)

In [ ]:
# dir = 'locations_train.csv'
# df = pd.read_csv(dir)
#
# file_paths = df['path'].values
# labels = df['grid'].values
# training_data =tf.data.Dataset.from_tensor_slices((file_paths, labels))
# def read_image(image_file, label):
#     image = tf.io.read_file(image_file)
#     image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
#     return image, label
#
# training_data = training_data.map(read_image).batch(64)

In [ ]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
# model.add(tf.keras.layers.Flatten())
#
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(training_data, epochs=10, verbose=2)